# Проверка последовательного анализа

Если вы хотите проверить реализацию последовательного анализа для *бинарных величин*,
вы можете воспользоваться этим блокнотом,
скопировав код из файлов `BinaryOneSampleSprt` и `BinaryTwoSampleSprt`.
Перебирая различные параметры можно проверить корректность подхода именно в вашем случае.

Отмечу, что обширная проверка корректности выполнялась в [checking](/binary/checking).

# Импорты функций

In [1]:
from scipy.stats import bernoulli

from binary.sprt import BinaryOneSampleSprt, BinaryTwoSampleSprt

## Случай двух выборок

In [2]:
p0 = 0.07
d = 0.005
alpha = 0.05
beta = 0.2
alternative = "greater"

# Количество тестов, которые моделируются независимо
iter_size = 100

In [3]:
p1 = p0 + d
p2 = p0
decision_stat = {}

for _ in range(iter_size):
    decision_desc = "Тест продолжается"
    sprt = BinaryTwoSampleSprt(p0, d, alpha, beta, 
                               alternative=alternative)
    
    while decision_desc == "Тест продолжается":
        # Определение выборки, из которой генерируется случайная величина
        sample_num = 1 + bernoulli.rvs(0.5)
        first_sample_flg = sample_num == 1

        if first_sample_flg:
            x = bernoulli.rvs(p1)
        else:
            x = bernoulli.rvs(p2)

        decision_desc = sprt.append(x, first_sample_flg)

        if decision_desc != "Тест продолжается":
            stop_flg = True
        
    decision_stat[decision_desc] = decision_stat.get(decision_desc, 0) + 1
    
for decision_desc in decision_stat:
    decision_freq = decision_stat[decision_desc] / iter_size
    print(f"Частота принятия решения '{decision_desc}' равна {decision_freq:.2%}.")

Частота принятия решения 'Тест остановлен, справедлива альтернатива p1 > p2' равна 82.00%.
Частота принятия решения 'Тест остановлен, справедлива гипотеза p1 <= p2' равна 18.00%.


Альтернатива `greater` состоит в том, что вероятность успеха (конверсия) первой выборки `p1`
больше, чем вероятность успеха (конверсия) второй выборки `p2`.
Так как `p1 = p0 + d`, а `p2 = p0`, то справедлива альтернатива,
и стоит ожидать, что вероятность отклонения гипотезы будет примерно равна `1 - beta = 80%`.
Как оказалось, это примерно так и есть.

## Случай одной выборки

In [6]:
p0 = 0.07
d = 0.005
alpha = 0.05
beta = 0.2
alternative = "two-sided"

# Количество тестов, которые моделируются независимо
iter_size = 500

In [7]:
p = p0
decision_stat = {}

for _ in range(iter_size):
    decision_desc = "Тест продолжается"
    sprt = BinaryOneSampleSprt(p0, d, alpha, beta, 
                               alternative=alternative)
    
    while decision_desc == "Тест продолжается":
        x = bernoulli.rvs(p)
        decision_desc = sprt.append(x)

        if decision_desc != "Тест продолжается":
            stop_flg = True
        
    decision_stat[decision_desc] = decision_stat.get(decision_desc, 0) + 1
    
for decision_desc in decision_stat:
    decision_freq = decision_stat[decision_desc] / iter_size
    print(f"Частота принятия решения '{decision_desc}' равна {decision_freq:.2%}.")

Частота принятия решения 'Тест остановлен, справедлива гипотеза p = p0' равна 93.40%.
Частота принятия решения 'Тест остановлен, справедлива альтернатива p < p0' равна 4.20%.
Частота принятия решения 'Тест остановлен, справедлива альтернатива p > p0' равна 2.40%.


Альтернатива `two-sided` состоит в том, что вероятность успеха (конверсия) `p`
отлична от заранее заданного значения `p0`.
Так как `p = p0`, то справедлива гипотеза,
и стоит ожидать, что вероятность отклонения гипотезы будет примерно равна `alpha = 5%`.
Как оказалось, это примерно так и есть.